# Table Creation
Let's get some data into some SQL tables so we can use it.

In [115]:
# imports
import pandas as pd
import sqlite3 as sql

import os
import nflgame as nfl

from collections import defaultdict

In [70]:
con = sql.Connection("/Users/tstuessi/Dev/ff-analysis/fantasyfootball")

In [64]:
files = [
    "CBS_YEAR_2014",
    "CBS_YEAR_2015",
    "CBS_YEAR_2016",
    "CBS_YEAR_2017",
    "FLEAFLICKER_YEAR_2014",
    "FLEAFLICKER_YEAR_2015",
    "FLEAFLICKER_YEAR_2016",
    "FLEAFLICKER_YEAR_2017"
]

In [85]:
con = sql.Connection("/Users/tstuessi/Dev/ff-analysis/fantasyfootball.db")
for file in files:
    df = pd.read_csv("{}.csv".format(file), index_col="playerid")
    df["total"] = df.sum(axis=1)
    df.reset_index().to_sql(file, con, if_exists="replace",index=False)
con.close()

## Create the player stats
The db is strange in that it cannot show trades between teams and only updates to the most recent roster; we might have to do some manual investigation on that front.

In [88]:
con = sql.Connection("/Users/tstuessi/Dev/ff-analysis/fantasyfootball.db")
for year in range(2014, 2018, 1):
    year_df = pd.DataFrame()
    i = 0
    for week in range(1,18, 1):
        games = nfl.games(year, week=week)
        players = nfl.combine_game_stats(games)
        
        dict_list = {}
        for p in players:
            tmp_dict = p.__dict__
            tmp_dict["position"] = p.__dict__["player"].position
            tmp_dict.pop("_stats", None)
            tmp_dict.pop("player", None)
            dict_list[i] = tmp_dict
            i += 1
        
        df = pd.DataFrame.from_dict(dict_list, orient="index")
        df["week"] = week
        year_df = year_df.append(df)
    
    year_df.fillna(0).to_sql("PLAYER_WEEKLY_STATS_YEAR_{}".format(year), con, if_exists="replace", index=False)
con.close()

/Users/tstuessi/anaconda3/envs/ff_analysis/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


## Create team position stats

In [191]:
con = sql.Connection("/Users/tstuessi/Dev/ff-analysis/fantasyfootball.db")
for year in range(2014, 2018, 1):
    week_by_week_df = pd.read_sql("SELECT * FROM PLAYER_WEEKLY_STATS_YEAR_{}".format(year), con, index_col="playerid")
    grouped = week_by_week_df.drop("week", axis=1).groupby(["team", "position"]).sum()
    unstacked = grouped.unstack()
    unstacked.columns = unstacked.columns.map("{0[1]}_{0[0]}".format)
    s_un = unstacked.reindex_axis(sorted(unstacked.columns),axis=1)
    s_un.fillna(0).to_sql("TEAM_POSITION_STATS_{}".format(year), con, index=True, index_label="team", if_exists="replace")
con.close()

/Users/tstuessi/anaconda3/envs/ff_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  import sys


In [190]:
# I bet we can do a bit better than just summing up the total things -- that's not a good measurement.
# let's do stats sums for each position for each player; this will be easier to do in pandas.
con = sql.Connection("/Users/tstuessi/Dev/ff-analysis/fantasyfootball.db")
for year in range(2014, 2018, 1):
    week_by_week_df = pd.read_sql("SELECT * FROM PLAYER_WEEKLY_STATS_YEAR_{}".format(year), con, index_col="playerid")
    per_player_totals_df = week_by_week_df.drop(["week"], axis=1).groupby(["playerid", "team", "position"]).sum().reset_index()

        # grab the team stats
    team_stats = week_by_week_df.drop("week", axis=1).groupby(["team", "position"]).sum()
    unstacked = team_stats.unstack()
    unstacked.columns = unstacked.columns.map("{0[1]}_{0[0]}".format)
    s_un = unstacked.reindex_axis(sorted(unstacked.columns),axis=1)
        # join these team stats
    joined_df = per_player_totals_df.merge(s_un, left_on="team", right_index=True)
    joined_df.fillna(0, inplace=True)

    for i, row in joined_df.iterrows():
        team = row["team"]
        position = row["position"]

        column_list = [x for x in row.index.values if x.startswith("{}_".format(position))]
        if len(column_list) == 0:
            continue
        orig_list = [x.replace("{}_".format(position), "") for x in column_list]

        players_stats = row[orig_list]
        team_stats = row[column_list]
        #print((team_stats - players_stats))

        row.loc[column_list] = team_stats - players_stats
    joined_df.to_sql("FULL_YEAR_PLAYER_AND_TEAM_STATS_{}".format(year), con, index=False)

con.close()

/Users/tstuessi/anaconda3/envs/ff_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  if sys.path[0] == '':
/Users/tstuessi/anaconda3/envs/ff_analysis/lib/python3.6/site-packages/pandas/core/series.py:851: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [188]:
joined_df.loc[0, :]
#con.close()

/Users/tstuessi/anaconda3/envs/ff_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  if sys.path[0] == '':
/Users/tstuessi/anaconda3/envs/ff_analysis/lib/python3.6/site-packages/pandas/core/series.py:851: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


playerid             00-0004091
team                        ARI
position                      K
defense_ast                   0
defense_ffum                  0
defense_int                   0
defense_sk                    0
defense_tkl                   0
fumbles_lost                  0
fumbles_rcv                   0
fumbles_tot                   0
fumbles_trcv                  0
fumbles_yds                   0
games                        16
home                          8
kicking_fga                  38
kicking_fgm                  32
kicking_fgyds               526
kicking_totpfg               96
kicking_xpa                  24
kicking_xpb                   2
kicking_xpmade               23
kicking_xpmissed              1
kicking_xptot                23
kickret_avg                   0
kickret_lng                   0
kickret_lngtd                 0
kickret_ret                   0
kickret_tds                   0
passing_att                   0
                        ...    
_passing

## Create Schedule Data

In [124]:
con = sql.Connection("/Users/tstuessi/Dev/ff-analysis/fantasyfootball.db")
for year in range(2014, 2019, 1):
    games_dict = {}
    games = nfl.games(year)
    for i in range(0, len(games)*2, 2):
        game = games[i // 2]
        games_dict[i] = {"team": game.home,
                         "week": game.schedule["week"],
                         "is_home": True,
                         "opp": game.away,
                         "team_score": game.score_home,
                         "opp_score": game.score_away,
                         "month": game.schedule["month"],
                         "wday": game.schedule["wday"]
                        }
        games_dict[i+1] = {"team": game.away,
                           "week": game.schedule["week"],
                           "is_home": False,
                           "opp": game.home,
                           "team_score": game.score_away,
                           "opp_score": game.score_home,
                           "month": game.schedule["month"],
                           "wday": game.schedule["wday"]
                          }
                           
    df = pd.DataFrame.from_dict(games_dict, orient="index")
    df.to_sql("SCHED_YEAR_{}".format(year), con, index=False)
    
    
con.close()

TypeError: 'NoneType' object is not iterable